In [2]:
import sys
sys.path.insert(0, '../../dataset/Dataset Processing/')
# sys.path.insert(0, '../Construct Strike Algorithm/')

In [3]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import time
import heapq
from data_processing import *  

In [4]:
# def createGraph(depotNodes ,requiredEdges, numNodes, show=True):
#     G = nx.Graph()
#     edges = []
#     pos = {}
#     reqPos = {}
#     s = [1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 7]
#     t = [2, 3, 4, 6, 4, 5, 5, 7, 6, 8, 7, 8]
#     weights = [2.3, 2, 3, 1.5, 3.2, 2.2, 3.8, 2.6, 2.2, 2.8, 1.8, 0.8]
#     xData = [-2, -0.5, -1,   0, 1,  1.5, 2,   2.5];
#     yData = [ 0, -2,    2.5, 0, 3, -2,   0.3, 1.5];
#     for i in range(len(s)):
#         edges.append((s[i], t[i], weights[i]))
    
#     for i in range(1, numNodes+1):
#         G.add_node(i)
#         pos[i] =(xData[i-1], yData[i-1])
    
#     node_color = ['y']*int(G.number_of_nodes())
#     depot_node_color = node_color
#     for i in range(1, len(node_color)+1):
#         if i in depotNodes:
#             depot_node_color[i-1] = 'g'
            
#     G.add_weighted_edges_from(edges)
#     labels = nx.get_edge_attributes(G,'weight')
#     nx.draw_networkx(G,pos, node_color = node_color)
#     nx.draw_networkx(G,pos, node_color = depot_node_color)
#     nx.draw_networkx_edges(G, pos, edgelist=requiredEdges, width=3, alpha=0.5,
#                                         edge_color="r")
#     nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
#     if show:
#         plt.figure(1)
#         plt.show()
#     return G,pos, depot_node_color

In [5]:
# Allocating task based on distance between base station and desired edge and UAV availability
def taskAllocation(G, depotNodes, requiredNodes, desiredEdgeTravesed, numrequiredEdges, uavsInDepotNodes):
    uavsInDepotNodesCopy = uavsInDepotNodes.copy()
    depotNodesCost = np.zeros((len(depotNodes), numrequiredEdges))
    depotPath = []
    bestPathTillDesiredEdge = []
    bestCostTillDesiredEdge = []
    if not all(uav == 0 for uav in uavsInDepotNodesCopy):
        for j in range(numrequiredEdges):
            if not desiredEdgeTravesed[j]:
                for i in range(len(depotNodes)):
                    if uavsInDepotNodesCopy[i] > 0:
                            c1 = nx.dijkstra_path_length(G, source=depotNodes[i], target=requiredNodes[j][0])
                            c2 = nx.dijkstra_path_length(G, source=depotNodes[i], target=requiredNodes[j][1])
                            c2 = np.inf
                            l = []
                            if c1 <= c2:
                                l = nx.dijkstra_path(G, source=depotNodes[i], target=requiredNodes[j][0])
                                l.append(requiredNodes[j][1])
                            else:
                                l = nx.dijkstra_path(G, source=depotNodes[i], target=requiredNodes[j][1])
                                l.append(requiredNodes[j][0])
                            depotNodesCost[i,j] = min(c1,c2)
                            depotNodesCost[i,j] += G.get_edge_data(requiredNodes[j][0], requiredNodes[j][1])['weight']
                            depotPath.append(l)
                    else:
                        depotNodesCost[i,j] = np.inf
                        depotPath.append([])
            else:
                depotNodesCost[:,j] = np.inf
                for i in range(len(depotNodes)):
                    depotPath.append([])

            if uavsInDepotNodesCopy[np.argmin(depotNodesCost[:,j])] > 0 and depotNodesCost[np.argmin(depotNodesCost[:,j]), j] != np.inf:
                uavsInDepotNodesCopy[np.argmin(depotNodesCost[:,j])] -= 1
            else:
                depotNodesCost[np.argmin(depotNodesCost[:,j]),j] = np.inf
#         if any(path for path in depotPath):
        depotPath = np.transpose(np.array(depotPath, dtype=object).reshape((numrequiredEdges, len(depotNodes))))
#         else:
#             depotPath = np.zeros((numrequiredEdges, len(depotNodes)))
        taskAllocatedtoBaseStations = []
        
        
        print("Task Allocation Algorithm Output: ")
        for i in range(numrequiredEdges):
            if not desiredEdgeTravesed[i]:
                if depotNodesCost[np.argmin(depotNodesCost[:,i]), i] != np.inf:
                    taskAllocatedtoBaseStations.append(np.argmin(depotNodesCost[:,i]))
                    bestCostTillDesiredEdge.append(depotNodesCost[taskAllocatedtoBaseStations[i],i])
                    bestPathTillDesiredEdge.append(depotPath[taskAllocatedtoBaseStations[i],i])
                    print('Allocating arc ' + str(requiredNodes[i][0]) + ' - ' + str(requiredNodes[i][1]) + ' to base station - node ' + str(depotNodes[taskAllocatedtoBaseStations[i]]))
                else:
                    taskAllocatedtoBaseStations.append(None)
                    bestCostTillDesiredEdge.append(None)
                    bestPathTillDesiredEdge.append(None)
                    print('Could not allocate edge ' + str(requiredNodes[i][0]) + ' - ' + str(requiredNodes[i][1]) + ' to any base stations')
            else:
                taskAllocatedtoBaseStations.append('Task Completed')
                bestCostTillDesiredEdge.append('Task Completed')
                bestPathTillDesiredEdge.append('Task Completed')
        
    else:
        print('No UAVs available. Task allocation not possible')
        
    return bestPathTillDesiredEdge, bestCostTillDesiredEdge

In [7]:
def pathScanningAlgorithm(G, numrequiredEdges, depotNodes, uavsInDepotNodes,
                          desiredEdgeTravesed, uavs, bestPathTillDesiredEdge,
                          bestCostTillDesiredEdge, vehicleCapacity, rechargeTime, numRecharge):
    
    bestRoute = []
    bestRouteCost = []
    minCost = np.inf
    
    for j in range(numrequiredEdges):
        if not desiredEdgeTravesed[j] and bestPathTillDesiredEdge[j] != None:
            minCost = np.inf
            l = []
            for i in range(len(depotNodes)):
                c1 = nx.dijkstra_path_length(G, source=bestPathTillDesiredEdge[j][-1], target=depotNodes[i])
                if c1 <= minCost:
                    l = nx.dijkstra_path(G, source=bestPathTillDesiredEdge[j][-1], target=depotNodes[i])[1:]
                    minCost = c1
            bestRoute.append(bestPathTillDesiredEdge[j] + l)
            bestRouteCost.append(bestCostTillDesiredEdge[j] + minCost)
            if bestRouteCost[j] > vehicleCapacity:
                bestRoute[j] = None
                bestRouteCost[j] = None
            else:
                desiredEdgeTravesed[j] = True
                if uavsInDepotNodes[depotNodes.index(bestRoute[j][0])] != 0: 
                    uavsInDepotNodes[depotNodes.index(bestRoute[j][0])] -= 1
                
                if heapq.heappop(uavs[depotNodes.index(bestRoute[j][0])]) == rechargeTime:
                    numRecharge += 1
                
                uavsInDepotNodes[depotNodes.index(bestRoute[j][-1])] += 1
                heapq.heappush(uavs[depotNodes.index(bestRoute[j][-1])], rechargeTime)
        else:
            bestRoute.append(None)
            bestRouteCost.append(None)
    print("Path Scanning Algorithm Output: ")
    return bestRoute, bestRouteCost, uavs, desiredEdgeTravesed, uavsInDepotNodes, numRecharge

In [14]:
def visualizePath(G, pos, Node_color, depot_node_color, Edges, depotNodes, requiredNodes, numNodes, path, pathType="solution"):
    for j in range(len(path)):
        if path[j] is not None and path[j] != 'Task Completed':
            plt.figure(1)
    
            G1 = nx.DiGraph()
            pos1 = {}
            node_color = []
            edges = []
            for i in range(len(path[j])-1):
                edges.append((path[j][i], path[j][i+1], G.get_edge_data(path[j][i], path[j][i+1])['weight']))
                pos1[path[j][i]] = pos[path[j][i]]
                if i == len(path[j])-2:
                    pos1[path[j][i+1]] = pos[path[j][i+1]]

            for key in pos1.keys():
                node_color.append(depot_node_color[key-1])

            G1.add_weighted_edges_from(edges)
            G.add_weighted_edges_from(Edges)
            labels = nx.get_edge_attributes(G,'weight')
            nx.draw_networkx(G,pos, node_color = Node_color)
            nx.draw_networkx(G,pos, node_color = depot_node_color)
            nx.draw_networkx_edges(G, pos, edgelist=requiredNodes, width=3, alpha=0.5,
                                                edge_color="r")
            nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
            nx.draw_networkx(G1,pos1, arrows=True, node_color = node_color, edge_color='b', arrowsize=12, width=1, arrowstyle='simple')
            if pathType == "solution":
                plt.legend(["Solution Path"], loc ="upper left")
            else:
                plt.legend(["Path"], loc ="upper left")
            plt.show()
        else:
            continue 

In [15]:
def createAndSaveGraph(inputType = 'txt'):
    info = readAndStoreInstanceInfo('../../dataset/CARP_datasets/DeArmon_gdb-IF')
    folderPath = '../../dataset/CARP_datasets/DeArmon_gdb-IF'
    savePath = '../../dataset/graph files/'
    vehicleCapacity = [0]*23
    numNodes = [0]*23
    requiredNodes = [0]*23
    depotNodes = [0]*23
    uavsInDepotNodes = [0]*23
    rechargeTime = [0]*23
    edges = [0]*23
    node_color = [0]*23
    depot_node_color = [0]*23
    pos = [0]*23
#     folderPath = '../CARP_datasets/DeArmon_gdb-IF'
#     for i, file in enumerate(os.listdir(folderPath)):
#         if file.endswith(".txt"):
#             file_path = f"{folderPath}/{file}"

    for i, file in enumerate(os.listdir(folderPath)):
        if file.endswith(".txt"):
            index = int(file[:-4].split('-')[-1])-1
            file_path = f"{folderPath}/{file}"
            startNode, endNode, edgeWeight = createGraphfromFile(file_path, info, i)
            G,pos[index], node_color[index], depot_node_color[index], edges[index], depotNodes[index], requiredNodes[index], vehicleCapacity[index], numNodes[index] = creatingIcyRoadInstance(file_path, info, i, startNode, endNode, edgeWeight)
            nx.write_gpickle(G, savePath + str(index) + '.pkl')

    np.save('../../dataset/graph info/pos.npy', np.array(pos, dtype=object))
    np.save('../../dataset/graph info/node_color.npy', np.array(node_color, dtype=object))
    np.save('../../dataset/graph info/depot_node_color.npy', np.array(depot_node_color, dtype=object))
    np.save('../../dataset/graph info/edges.npy', np.array(edges, dtype=object))
    np.save('../../dataset/graph info/depotNodes.npy', np.array(depotNodes, dtype=object))
    np.save('../../dataset/graph info/requiredNodes.npy', np.array(requiredNodes, dtype=object))
    np.save('../../dataset/graph info/vehicleCapacity.npy', np.array(vehicleCapacity, dtype=object))
    np.save('../../dataset/graph info/numNodes.npy', np.array(numNodes, dtype=object))
    
    return info
#     file_path = '../../dataset/CARP_datasets/DeArmon_gdb-IF/gdb-IF-01.txt'
#     startNode, endNode, edgeWeight = createGraphfromFile(file_path, info, 0)
#     G,pos, node_color, depot_node_color, edges, depotNodes, requiredNodes, vehicleCapacity, numNodes = creatingIcyRoadInstance(file_path, info, 0, startNode, endNode, edgeWeight)

In [16]:
def main():
    # Initializing Parameters
#     vehicleCapacity = 14
#     numNodes = 8
#     requiredNodes = [[2, 4], [6, 7], [3, 4]]
#     depotNodes = [1, 5]
#     uavsInDepotNodes = [1, 1]
#     rechargeTime = 20
      
    position = list(np.load('../../dataset/graph info/pos.npy', allow_pickle=True))
    nodeColor = list(np.load('../../dataset/graph info/node_color.npy', allow_pickle=True))
    depotNodeColor = list(np.load('../../dataset/graph info/depot_node_color.npy', allow_pickle=True))
    Edges = list(np.load('../../dataset/graph info/edges.npy', allow_pickle=True))
    vehCap = list(np.load('../../dataset/graph info/vehicleCapacity.npy', allow_pickle=True))
    nuNod = list(np.load('../../dataset/graph info/numNodes.npy', allow_pickle=True))
    reqNod = list(np.load('../../dataset/graph info/requiredNodes.npy', allow_pickle=True))
    deNo = list(np.load('../../dataset/graph info/depotNodes.npy', allow_pickle=True))
    
    info = readAndStoreInstanceInfo('../../dataset/CARP_datasets/DeArmon_gdb-IF')
    instanceData = {'Instance Name' : [],
                    'Number of Nodes' : [],
                    'Number of Edges' : [],
                    'Number of Required Edges' : [],
                    'Capacity' : [],
                    'Number of Depot Nodes' : []}

    algoOutputData = {'Execution Time' : [],
                      'Maximum Cost' : [],
                      'Total Cost' : [],
                      'Number of Recharges' : [],
                      'Number of Desired Edges Traversed' : [],
                      'Success Rate' : []}
    
    folderPath = '../../dataset/graph files'
    for i, file in enumerate(os.listdir(folderPath)):
        if file.endswith(".pkl"):
            index = int(file.split('.')[0])
            file_path = f"{folderPath}/{file}"
            
            G = nx.read_gpickle(file_path)
            pos = position[index]
            node_color = nodeColor[index]
            depot_node_color = depotNodeColor[index]
            edges = Edges[index]
            depotNodes = deNo[index]
            requiredNodes = reqNod[index]
            vehicleCapacity = vehCap[index]
            numNodes = nuNod[index]
            uavsInDepotNodes = [len(requiredNodes)//(2*len(depotNodes))]*len(depotNodes)
            rechargeTime = 2*vehicleCapacity
            instanceData['Instance Name'].append(info['Instance Name'][index])
            instanceData['Number of Nodes'].append(numNodes)
            instanceData['Number of Edges'].append(len(edges))
            instanceData['Number of Required Edges'].append(len(requiredNodes))
            instanceData['Capacity'].append(vehicleCapacity)
            instanceData['Number of Depot Nodes'].append(len(depotNodes))
            
#     G,pos, node_color, depot_node_color, edges, depotNodes, requiredNodes, vehicleCapacity, numNodes = createGraph()
#     uavsInDepotNodes = [len(requiredNodes)//(2*len(depotNodes))]*len(depotNodes)
#     rechargeTime = 2*vehicleCapacity
    
            # Declaring useful paramerters
            numrequiredEdges = len(requiredNodes)
            desiredEdgeTravesed = [False]*numrequiredEdges
            numRecharge = 0
            iterations = 0
            uavs = []
            allRoutes = []
            allRoutesCosts = []
            for element in uavsInDepotNodes:
                l = [0]*element
                heapq.heapify(l)
                uavs.append(l)
            penality = 0
            print(sum(uavsInDepotNodes))
            start = time.time()
        #     G,pos, depot_node_color = createGraph(depotNodes, requiredNodes, numNodes, show=False)

            while not all(edges == True for edges in desiredEdgeTravesed):
                if iterations <= sum(uavsInDepotNodes)*numrequiredEdges:
                    bestPathTillDesiredEdge, bestCostTillDesiredEdge = taskAllocation(G, depotNodes, requiredNodes, desiredEdgeTravesed, numrequiredEdges, uavsInDepotNodes)
            #         visualizePath(depotNodes, requiredNodes, numNodes, bestPathTillDesiredEdge, pathType="normal")
#                     bestRoute, bestRouteCost, uavs, desiredEdgeTravesed, uavsInDepotNodes, numRecharge = pathScanningAlgorithm(G, numrequiredEdges,
#                                                                                                                                  depotNodes, uavsInDepotNodes,
#                                                                                                                                  desiredEdgeTravesed, uavs,
#                                                                                                                                  bestPathTillDesiredEdge, bestCostTillDesiredEdge,
#                                                                                                                                  vehicleCapacity, rechargeTime, numRecharge)
                
                    G, bestRoute, bestRouteCost, uavs, desiredEdgeTravesed, uavsInDepotNodes, numRecharge = constructAndStrikeAlgorithm(G, numrequiredEdges,
                                                                                                                                     depotNodes, uavsInDepotNodes,
                                                                                                                                     desiredEdgeTravesed, uavs,
                                                                                                                                     bestPathTillDesiredEdge, bestCostTillDesiredEdge,
                                                                                                                                     vehicleCapacity, rechargeTime, numRecharge)
                
            #         visualizePath(depotNodes, requiredNodes, numNodes, bestRoute)
                    for i in range(len(bestRoute)):
                        if bestRoute[i] != None:
                            allRoutes.append(bestRoute[i])
                            allRoutesCosts.append(bestRouteCost[i])
                else:
                    break
                iterations += 1

            end = time.time()
        #     visualizePath(depotNodes, requiredNodes, numNodes, allRoutes)
            if i == 10:
                print(instanceData['Instance Name'][-1])
#                 visualizePath(G,pos, node_color, depot_node_color, edges, depotNodes, requiredNodes, numNodes, allRoutes)
            print(str(desiredEdgeTravesed.count(True)) + ' out of '+ str(len(desiredEdgeTravesed)) + ' is Traversed.')
            print('Success rate is ' + str((desiredEdgeTravesed.count(True)/len(desiredEdgeTravesed))*100) + '%.')
            print('Maximum of all path cost = ' + str(max(allRoutesCosts)))
            print('Number of times drones had to recharge = ' + str(numRecharge))
            print('Total cost of all paths + recharge cost = ' + str(sum(allRoutesCosts) + rechargeTime*numRecharge))
            print("Execution took "+ str(end-start) + " seconds.")
            algoOutputData['Execution Time'].append(round(end-start, 3))
            algoOutputData['Maximum Cost'].append(max(allRoutesCosts))
            algoOutputData['Total Cost'].append(sum(allRoutesCosts) + rechargeTime*numRecharge)
            algoOutputData['Number of Recharges'].append(numRecharge)
            algoOutputData['Number of Desired Edges Traversed'].append(desiredEdgeTravesed.count(True))
            algoOutputData['Success Rate'].append((desiredEdgeTravesed.count(True)/len(desiredEdgeTravesed))*100)
            print()
            print()
#             df = pd.DataFrame(instanceData)
#             df.to_csv('../../dataset/dataset_info.csv')
            
            df = pd.DataFrame(algoOutputData)
            df.to_csv('../../dataset/algo_output_info.csv')
            
if __name__ == "__main__":
    # execute only if run as a script
    main()

Index(['Instance Name', 'Number of Nodes', 'Number of Edges', 'Required Edges',
       'Capacity', 'Number of Depot Nodes', 'Depot Nodes'],
      dtype='object')
3
Task Allocation Algorithm Output: 
Allocating arc 1 - 2 to base station - node 1
Allocating arc 1 - 10 to base station - node 9
Allocating arc 2 - 9 to base station - node 11
Could not allocate edge 3 - 4 to any base stations
Could not allocate edge 5 - 11 to any base stations
Could not allocate edge 7 - 8 to any base stations
Could not allocate edge 7 - 12 to any base stations
Could not allocate edge 8 - 11 to any base stations
Construct Strike Algorithm Output: 
Task Allocation Algorithm Output: 
Allocating arc 1 - 10 to base station - node 9
Allocating arc 3 - 4 to base station - node 9
Allocating arc 5 - 11 to base station - node 9
Could not allocate edge 7 - 8 to any base stations
Could not allocate edge 7 - 12 to any base stations
Could not allocate edge 8 - 11 to any base stations
Construct Strike Algorithm Output: 
T

TypeError: 'NoneType' object is not subscriptable